In [ ]:
# This is another way to pull in your data for analysis using SQL by calling your query results
%%bigquery results --project prod-organize-arizon-4e1c0a83
SELECT
  p.pctnum
  , d.countyname
  , n.race
  , d.congressionaldistrict
  , d.statehousedistrict
  , COUNT(DISTINCT n.dwid) as people_count

FROM `proj-tmc-mem-mvp.catalist_enhanced.enh_catalist__ntl_current` AS n

LEFT JOIN `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS d
  ON n.dwid = d.dwid

INNER JOIN `prod-organize-arizon-4e1c0a83.rich_christina_proj.catalist_pctnum_crosswalk_native` AS p
  ON d.uniqueprecinctcode = p.uniqueprecinctcode

WHERE d.state = 'AZ'
  AND p.pctnum IS NOT NULL


GROUP BY 1,2,3,4,5

ORDER BY 1,3

In [ ]:
bq_df = results

In [ ]:
import bigframes.pandas as bf

df = bq_df
df.head(10)

In [ ]:
pivot_df = df.pivot_table(
    index=['pctnum', 'countyname', 'congressionaldistrict', 'statehousedistrict'],
    columns='race',
    values='people_count',
    fill_value=0
).astype(int).reset_index()

pivot_df.head()

In [ ]:
from pandas_gbq import to_gbq


project_id = "prod-organize-arizon-4e1c0a83"
dataset_table = "rich_christina_proj.race_counts_pivot"  # dataset.table

# Upload DataFrame to BigQuery
to_gbq(
    pivot_df,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists='replace'  # or 'append'
)
